## ISI Epilepsy Brno & Montreal patients with H2o AutoML framework

<u>**Goal:**</u>: Using H2o AutoML, predict the "onset channel" of patients from Brno and Montreal based on the selected features, by considering only outcome # 11 and segm_type # 4, applying a "leave one patient out" method.

15/2/2023

---

In [1]:
# Import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

In [2]:
# Import dataset

df = pd.read_csv('~/Marek/2_PhD/ISI/Work/data/new_isi_data.csv')
# Remove unnamed column: ,index_col=0
df.head()

,Unnamed: 0,patient_id,channel_name,resected,onset_channel,outcome,HFO_80.0-250.0,HFO_250.0-600.0,spike_rate,power_1.0-4.0,...,lin_corr_median_80.0-250.0,ren_80.0-250.0,phase_sync_80.0-250.0,coherence_80.0-250.0,lin_corr_median_250.0-600.0,ren_250.0-600.0,phase_sync_250.0-600.0,coherence_250.0-600.0,segm_type,segm_number
0,0,583,ra1,0,NON_SOZ,11.0,11,34,22,0.533375,...,0.995615,0.003025,0.985229,0.997413,0.990955,0.004121,0.970733,0.994151,0,0
1,1,583,ra2,0,NON_SOZ,11.0,16,40,49,0.518898,...,0.999169,0.000649,0.996365,0.999503,0.997006,0.001574,0.987965,0.997992,0,0
2,2,583,ra3,0,NON_SOZ,11.0,20,35,61,0.466505,...,0.999172,0.000648,0.996397,0.999509,0.996981,0.001719,0.987949,0.997969,0,0
3,3,583,ra4,0,NON_SOZ,11.0,27,37,66,0.444008,...,0.997592,0.001411,0.991613,0.998744,0.994820,0.002903,0.984822,0.997324,0,0
4,4,583,ra5,0,NON_SOZ,11.0,11,58,63,0.588096,...,0.994437,0.003369,0.981503,0.996723,0.996824,0.001831,0.987556,0.997855,0,0


In [3]:
# Drop index column

df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.head()

,patient_id,channel_name,resected,onset_channel,outcome,HFO_80.0-250.0,HFO_250.0-600.0,spike_rate,power_1.0-4.0,power_4.0-8.0,...,lin_corr_median_80.0-250.0,ren_80.0-250.0,phase_sync_80.0-250.0,coherence_80.0-250.0,lin_corr_median_250.0-600.0,ren_250.0-600.0,phase_sync_250.0-600.0,coherence_250.0-600.0,segm_type,segm_number
0,583,ra1,0,NON_SOZ,11.0,11,34,22,0.533375,0.781969,...,0.995615,0.003025,0.985229,0.997413,0.990955,0.004121,0.970733,0.994151,0,0
1,583,ra2,0,NON_SOZ,11.0,16,40,49,0.518898,0.780688,...,0.999169,0.000649,0.996365,0.999503,0.997006,0.001574,0.987965,0.997992,0,0
2,583,ra3,0,NON_SOZ,11.0,20,35,61,0.466505,0.740345,...,0.999172,0.000648,0.996397,0.999509,0.996981,0.001719,0.987949,0.997969,0,0
3,583,ra4,0,NON_SOZ,11.0,27,37,66,0.444008,0.721173,...,0.997592,0.001411,0.991613,0.998744,0.994820,0.002903,0.984822,0.997324,0,0
4,583,ra5,0,NON_SOZ,11.0,11,58,63,0.588096,0.937518,...,0.994437,0.003369,0.981503,0.996723,0.996824,0.001831,0.987556,0.997855,0,0


In [4]:
# Drop "nan" channels across all segments

df_nans = df[df.isnull().any(axis=1)]
for pts in df_nans.patient_id.unique():
    for chns in list(df_nans.loc[df_nans.patient_id == pts,'channel_name'].unique()):
        df = df.drop(df[(df.patient_id == pts) & (df.channel_name == chns)].index)

In [5]:
# Dataframe preparation, target column 

outcome_11 = 11

df = df[df.outcome <= outcome_11]
mask = df.onset_channel == 'NON_SOZ'
df.loc[mask, 'onset_channel'] = 0
mask = df.onset_channel == 'SOZ'
df.loc[mask, 'onset_channel'] = 1
tmp = df['resected'].values * df['onset_channel'].values
df['target'] = tmp.astype(int)
df = df.rename(columns={'spike_rate':'spike_rate_0.0-0.0',
                        'pse':'pse_0.0-0.0',
                        'pac':'pac_0.0-0.0',
                        'fac':'fac_0.0-0.0',
                        'lfr':'lfr_0.0-0.0'})

features = [x for x in df if any(c.isdigit() for c in x)]
features.sort()

In [6]:
# Drop no target pts

target = 'target'
for pts in df.patient_id.unique():
    if sum(df.loc[df.patient_id == pts,target]) == 0:
        #print('patient',str(pts),'does not have target - droping this patient')
        df = df[df.patient_id != pts]

In [7]:
# Keep only segm_type = 4 

df = df.loc[df['segm_type'] == 4]
print(df.shape)
df.head()

(1959, 68)


,patient_id,channel_name,resected,onset_channel,outcome,HFO_80.0-250.0,HFO_250.0-600.0,spike_rate_0.0-0.0,power_1.0-4.0,power_4.0-8.0,...,ren_80.0-250.0,phase_sync_80.0-250.0,coherence_80.0-250.0,lin_corr_median_250.0-600.0,ren_250.0-600.0,phase_sync_250.0-600.0,coherence_250.0-600.0,segm_type,segm_number,target
34674,583,ra1,0,0,11.0,19,56,50,0.526201,0.474988,...,0.008171,0.962278,0.992315,0.951701,0.015386,0.894617,0.968141,4,0,0
34675,583,ra2,0,0,11.0,35,62,86,0.560877,0.560381,...,0.001929,0.989986,0.998410,0.982513,0.007849,0.952962,0.988048,4,0,0
34676,583,ra3,0,0,11.0,36,66,101,0.558763,0.576171,...,0.002038,0.989737,0.998386,0.982065,0.007308,0.951814,0.987657,4,0,0
34677,583,ra4,0,0,11.0,47,66,102,0.554024,0.565439,...,0.005044,0.975965,0.995656,0.975849,0.009962,0.941062,0.984107,4,0,0
34678,583,ra5,0,0,11.0,32,75,96,0.649021,0.658895,...,0.009224,0.944790,0.987954,0.980963,0.007094,0.949647,0.986945,4,0,0


In [8]:
# Number of unique patients

print(len(df.patient_id.unique()))
patients = df.patient_id.unique()
patients = np.sort(patients)
patients

18


array([  61,   63,   71,   77,   82,   89,   93,  583,  657,  717,  723,
        953,  965, 1002, 1021, 1041, 1043, 1630])

In [9]:
# Scale selected features with standard scaler (z-score)

from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder        
scaler = StandardScaler()
for pts in df.patient_id.unique(): 
    df.loc[df.patient_id == pts,features] = scaler.fit_transform(df.loc[df.patient_id == pts,features])

In [10]:
# Show dataframe

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1959 entries, 34674 to 40452
Data columns (total 68 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   patient_id                   1959 non-null   int64  
 1   channel_name                 1959 non-null   object 
 2   resected                     1959 non-null   int64  
 3   onset_channel                1959 non-null   object 
 4   outcome                      1959 non-null   float64
 5   HFO_80.0-250.0               1959 non-null   float64
 6   HFO_250.0-600.0              1959 non-null   float64
 7   spike_rate_0.0-0.0           1959 non-null   float64
 8   power_1.0-4.0                1959 non-null   float64
 9   power_4.0-8.0                1959 non-null   float64
 10  power_8.0-12.0               1959 non-null   float64
 11  power_12.0-20.0              1959 non-null   float64
 12  power_20.0-45.0              1959 non-null   float64
 13  power_65.0-80

In [11]:
# Create a list of columns

column_names = df.columns.tolist()
print(column_names)

['patient_id', 'channel_name', 'resected', 'onset_channel', 'outcome', 'HFO_80.0-250.0', 'HFO_250.0-600.0', 'spike_rate_0.0-0.0', 'power_1.0-4.0', 'power_4.0-8.0', 'power_8.0-12.0', 'power_12.0-20.0', 'power_20.0-45.0', 'power_65.0-80.0', 'power_80.0-250.0', 'power_250.0-600.0', 'hlx_0.0-0.0', 'hlx_1.0-4.0', 'hlx_4.0-8.0', 'hlx_8.0-12.0', 'hlx_12.0-20.0', 'hlx_20.0-45.0', 'hlx_65.0-80.0', 'hlx_80.0-250.0', 'hlx_250.0-600.0', 'pse_0.0-0.0', 'pac_0.0-0.0', 'fac_0.0-0.0', 'lfr_0.0-0.0', 'lin_corr_median_0.0-0.0', 'ren_0.0-0.0', 'phase_sync_0.0-0.0', 'coherence_0.0-0.0', 'lin_corr_median_1.0-4.0', 'ren_1.0-4.0', 'phase_sync_1.0-4.0', 'coherence_1.0-4.0', 'lin_corr_median_4.0-8.0', 'ren_4.0-8.0', 'phase_sync_4.0-8.0', 'coherence_4.0-8.0', 'lin_corr_median_8.0-12.0', 'ren_8.0-12.0', 'phase_sync_8.0-12.0', 'coherence_8.0-12.0', 'lin_corr_median_12.0-20.0', 'ren_12.0-20.0', 'phase_sync_12.0-20.0', 'coherence_12.0-20.0', 'lin_corr_median_20.0-45.0', 'ren_20.0-45.0', 'phase_sync_20.0-45.0', 'coh

In [12]:
# Revised AutoML model

import h2o
from h2o.automl import H2OAutoML

# Initialize h2o
h2o.init()

# Convert pandas dataframe to H2OFrame
hf = h2o.H2OFrame(df)

# Convert patient_id & target variable to categorical
hf['target'] = hf['target'].asfactor()
hf['patient_id'] = hf['patient_id'].asfactor()

# Define target and feature variables
target = "target"
features = list(features)

# Convert numpy ndarray to list of strings
patients = [61, 63, 71, 77, 82, 89, 93, 583, 657, 717, 723, 953, 965, 1002, 1021, 1041, 1043, 1630]
patients = list(map(str, patients))

for patient in patients:
    # Create a patient mask to filter out the current patient
    patient_mask = hf['patient_id'] != patient
    
    # Create a new train and test dataframe with the patient left out
    train_df = hf[patient_mask]
    test_df = hf[hf['patient_id'] == patient]
    
    # Define target and feature variables
    y = target
    X = features
    
    # Run AutoML on the filtered train set
    aml = H2OAutoML(max_models=10, seed=1)
    aml.train(x=X, y=y, training_frame=train_df)
    
    # Get the best model
    best_model = aml.leader
    
    # Make predictions on the test set
    predictions = best_model.predict(test_df)
    
    # Evaluate the model performance
    try:
        performance = best_model.model_performance(test_df)
        print("Patient ID: ", patient)
        print("Model Leader: ", best_model.model_id)
        print("Predictions: ", predictions)
        print("Performance: ", performance)
        print("-----------------------------")
    except Exception as e:
        print("Error for patient id: ", patient)
        print(str(e))
        print("-----------------------------")


Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.17" 2022-10-18; OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu222.04); OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu222.04, mixed mode, sharing)
  Starting server from /home/marek/.local/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpkd0ni4nk
  JVM stdout: /tmp/tmpkd0ni4nk/h2o_marek_started_from_python.out
  JVM stderr: /tmp/tmpkd0ni4nk/h2o_marek_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Prague
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.4
H2O_cluster_version_age:,1 month and 9 days
H2O_cluster_name:,H2O_from_python_marek_71vbcl
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.744 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Patient ID:  61
Model Leader:  StackedEnsemble_BestOfFamily_1_AutoML_1_20230214_201018
Predictions:    predict        p0           p1
        0  0.996366  0.00363373
        0  0.999008  0.000992098
        0  0.972596  0.027404
        0  0.98343   0.0165697
        0  0.98362   0.0163799
        0  0.998749  0.00125051
        0  0.985796  0.0142044
        0  0.998683  0.00131714
        0  0.994383  0.00561664
        0  0.94918   0.0508203
[150 rows x 3 columns]

Performance:  ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.0549910637434622
RMSE: 0.23450173505426822
LogLoss: 0.1948963487633547
AUC: 0.8329393223010244
AUCPR: 0.18654098551896198
Gini: 0.6658786446020488
Nul

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Patient ID:  71
Model Leader:  StackedEnsemble_BestOfFamily_1_AutoML_3_20230214_201053
Predictions:    predict        p0          p1
        0  0.975723  0.0242769
        0  0.953024  0.0469762
        0  0.931079  0.0689213
        0  0.96877   0.0312304
        0  0.98292   0.0170802
        0  0.945709  0.0542908
        0  0.987122  0.0128784
        0  0.994539  0.00546091
        0  0.989467  0.0105327
        0  0.9474    0.0526004
[169 rows x 3 columns]

Performance:  ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.022270829312476006
RMSE: 0.149234142582976
LogLoss: 0.08204475655439525
AUC: 0.9536585365853659
AUCPR: 0.4409207385295162
Gini: 0.9073170731707318
Null degrees of freedom: 168
Residual degrees of freedom: 164
Null deviance: 51.07593539168546
Residual

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Patient ID:  82
Model Leader:  StackedEnsemble_AllModels_1_AutoML_5_20230214_201119
Predictions:    predict        p0          p1
        0  0.978177  0.0218229
        0  0.992948  0.00705233
        0  0.986294  0.0137056
        0  0.996661  0.00333879
        0  0.990388  0.0096124
        0  0.96849   0.0315101
        0  0.97809   0.0219104
        0  0.981704  0.018296
        0  0.988511  0.0114894
        0  0.989476  0.0105237
[85 rows x 3 columns]

Performance:  ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.12114531949894433
RMSE: 0.3480593620331801
LogLoss: 0.4627062060505273
AUC: 0.7186732186732187
AUCPR: 0.20951238394328878
Gini: 0.4373464373464373
Null degrees of freedom: 84
Residual degrees of freedom: 79
Null deviance: 69.78195415231883
Residual devia

...

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Patient ID:  657
Model Leader:  StackedEnsemble_AllModels_1_AutoML_9_20230214_201212
Predictions:    predict        p0          p1
        0  0.992572  0.00742824
        0  0.997442  0.00255821
        0  0.997535  0.00246538
        0  0.996586  0.00341402
        0  0.996745  0.00325471
        0  0.995917  0.00408341
        0  0.981445  0.018555
        0  0.989168  0.0108322
        1  0.327594  0.672406
        0  0.989127  0.0108729
[45 rows x 3 columns]

Performance:  ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.12650047516172006
RMSE: 0.3556690528591433
LogLoss: 0.34564531695048084
AUC: 0.8253968253968254
AUCPR: 0.15962732123036932
Gini: 0.6507936507936507
Null degrees of freedom: 44
Residual degrees of freedom: 36
Null deviance: 22.049312915498845
Residual

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Patient ID:  723
Model Leader:  StackedEnsemble_AllModels_1_AutoML_11_20230214_201241
Predictions:    predict        p0          p1
        0  0.957482  0.0425184
        0  0.988304  0.0116965
        0  0.96592   0.0340799
        0  0.934983  0.0650167
        0  0.979467  0.020533
        0  0.985166  0.0148344
        0  0.996497  0.00350255
        0  0.994716  0.00528421
        0  0.996828  0.00317195
        0  0.987306  0.0126939
[86 rows x 3 columns]

Performance:  ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.044341575108585336
RMSE: 0.2105743932879431
LogLoss: 0.13609683079771567
AUC: 0.9878787878787879
AUCPR: 0.9097788965311602
Gini: 0.9757575757575758
Null degrees of freedom: 85
Residual degrees of freedom: 79
Null deviance: 69.91399716741043
Residual d

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Patient ID:  965
Model Leader:  GBM_3_AutoML_13_20230214_201311
Predictions:    predict        p0          p1
        0  0.657217  0.342783
        0  0.918229  0.0817711
        0  0.630749  0.369251
        0  0.783722  0.216278
        0  0.789173  0.210827
        0  0.9229    0.0770997
        0  0.993083  0.00691731
        0  0.996303  0.00369742
        0  0.995432  0.00456758
        0  0.992476  0.00752398
[86 rows x 3 columns]

Performance:  ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.048484719278370884
RMSE: 0.22019245963104842
LogLoss: 0.15815903401578765
Mean Per-Class Error: 0.19076305220883533
AUC: 0.9156626506024096
AUCPR: 0.17971293092630514
Gini: 0.8313253012048192

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.463871956487407
       0    1    Error    

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Patient ID:  1021
Model Leader:  StackedEnsemble_AllModels_1_AutoML_15_20230214_201343
Predictions:    predict        p0          p1
        0  0.960473  0.0395268
        0  0.995856  0.0041442
        0  0.994753  0.00524665
        0  0.994566  0.00543355
        0  0.967631  0.0323688
        0  0.986025  0.0139749
        1  0.666365  0.333635
        0  0.996213  0.00378668
        0  0.986698  0.0133022
        0  0.986731  0.0132688
[59 rows x 3 columns]

Performance:  ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.16143616926561924
RMSE: 0.4017912010803861
LogLoss: 0.5607909545837896
AUC: 0.9119241192411923
AUCPR: 0.8474329422699414
Gini: 0.8238482384823846
Null degrees of freedom: 58
Residual degrees of freedom: 50
Null deviance: 105.29892314894904
Residual d

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Patient ID:  1043
Model Leader:  StackedEnsemble_BestOfFamily_1_AutoML_17_20230214_201414
Predictions:    predict        p0          p1
        1  0.501215  0.498785
        0  0.775138  0.224862
        0  0.980393  0.0196065
        0  0.994206  0.00579398
        0  0.995839  0.00416079
        0  0.996022  0.00397803
        0  0.980579  0.0194212
        0  0.908082  0.0919175
        0  0.952445  0.0475554
        0  0.912018  0.0879822
[71 rows x 3 columns]

Performance:  ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.15524459363202212
RMSE: 0.39401090547346795
LogLoss: 0.5856858960787791
AUC: 0.4444444444444445
AUCPR: 0.10442292315761179
Gini: -0.11111111111111105
Null degrees of freedom: 70
Residual degrees of freedom: 66
Null deviance: 57.21844622238898
Resid

Explanation:

1) General

The script defines a list of patient IDs and iterates over each patient, creating a "patient mask" to filter out the current patient and creating a new train and test dataset with the current patient left out. AutoML is then run on the filtered train set, with a maximum of 10 models being built and a seed of 1. The best model is then used to make predictions on the test set, and the performance of the model is evaluated using the model_performance function. If the evaluation is successful, the script prints the patient ID, the model ID of the best model, the predictions made by the model, and the performance metrics. If an error occurs during evaluation, the script prints an error message.

2) This code uses the following libraries:

h2o: a machine learning platform that provides an API for building, training, and deploying machine learning models.

H2OAutoML: a module in the h2o library that provides an automated machine learning interface for H2O, allowing users to train and select the best model for a given dataset without requiring extensive domain knowledge or expertise.


4) Cross-validation

A cross-validation loop that iterates through a list of patient IDs and performs AutoML on the dataset with each patient left out in turn. For each iteration, the code creates a mask to filter out the current patient from the dataset, and then creates new training and test dataframes without the current patient. The target variable and feature variables are defined, and then AutoML is run on the filtered training set to train and select the best model. By leaving out one patient at a time and training models on the remaining data, this code performs a form of cross-validation to evaluate the performance of the model on patients that were not used to train the model. This can help to detect overfitting and provide a more accurate estimate of the model's performance.


3)  An explanation of each evaluation metric:

The predictions made by the model are shown in a table that contains the predicted class (0 or 1), the predicted probability of class 0 (p0), and the predicted probability of class 1 (p1).

The performance of the model is evaluated using several metrics, including mean squared error (MSE), root mean squared error (RMSE), log loss, area under the receiver operating characteristic curve (AUC), area under the precision-recall curve (AUCPR), and Gini coefficient. 

Additionally, the confusion matrix is displayed, showing the number of true positives (TP), false positives (FP), true negatives (TN), and false negatives (FN) at the optimal threshold, which is the threshold that maximizes the F1 score. 

Finally, the gains/lift table is presented, which shows the lift in the response rate compared to the baseline response rate at different levels of score or probability thresholds.

---

Here are the most important information we can derive from them:

Confusion Matrix: It shows the actual and predicted values of the target variable. It helps in understanding how well the model is performing in terms of predicting the positive and negative classes.

AUC: Area Under the Receiver Operating Characteristic Curve (AUC-ROC) is a measure of how well the model is able to distinguish between the two classes. An AUC score of 1 means that the model is able to perfectly distinguish between the two classes, while a score of 0.5 indicates random guessing.

LogLoss: It is a measure of the model's performance in terms of the likelihood of the predicted probabilities. Lower log loss indicates better performance.

Maximum Metrics: These metrics help you identify the best threshold to use for making predictions, based on the metric you care about most (e.g., f1 score, recall, precision, etc.).

Gains/Lift Table: This table shows how well the model is able to capture the positive class, based on different thresholds. It is a useful tool to determine how well the model is performing at identifying the positive class and whether the model is better than randomly guessing.

Overall, a good model would have a high AUC, low LogLoss, and high values for the metrics that are most important for the task at hand (e.g., f1 score, precision, recall, etc.). It is also important to check the gains/lift table to ensure that the model is capturing the positive class effectively.